In [1]:
from pandas_datareader import data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
my_year_month_fmt = mdates.DateFormatter('%m/%y')

#import widgets packages
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import HBox,VBox,Layout
import datetime
import panel as pn
pn.extension()
import talib

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
from ta import add_all_ta_features

In [3]:
#get stock ticker and sector data
stock_df = pd.read_csv("./tickers.csv")

In [4]:
stock_df.head()

,Undervalued Growth Stocks,Undervalued Large Cap Stocks,Aggressive Small Cap Stocks,Growth Technology Stocks,Basic Materials Stocks,Consumer Cyclical,Financial Services,Real Estate,Consumer Defensive,Healthcare,Utilities,Communication Services,Energy,Industrials,Technology
0,GOLD,GM,SNDL,AMD,LIN,AMZN,HSBC-PA,AMT,WMT,JNJ,NEE,GOOGL,CVX,UPS,AAPL
1,KGC,GOLD,AHT,QCOM,BHP,BABA,BRK-A,BEKE,PG,UNH,DUK,GOOG,XOM,HON,MSFT
2,EBAY,RKT,AMC,CRM,BBL,TSLA,BRK-B,PLD,KO,PFE,DUK-PA,FB,RDS-A,UNP,TSM
3,BSX,HBAN,TLGT,VNET,CTA-PB,HD,V,CCI,PEP,MRK,D,VZ,RDS-B,LMT,NVDA
4,HWM,EBAY,HEXO,ZS,RIO,NKE,JPM,EQIX,UL,ABT,SO,DIS,PTR,BA,CRM


In [5]:
def get_data(data,start,end):
    # Getting just the adjusted closing prices. 
    #This will return a Pandas DataFrame
    # The index in this DataFrame is the major index 
    #of the panel_data.
    close = panel_data['Adj Close']

    # Getting all weekdays between 01/01/2000 and 12/31/2016
    all_weekdays = pd.date_range(start=start,\
                           end=end, freq='B')

    # How do we align the existing prices in adj_close 
    # with our new set of dates?
    # All we need to do is reindex close using all_weekdays
    # as the new index
    close = close.reindex(all_weekdays)

    # Reindexing will insert missing values (NaN) for the
    # dates that were not present
    # in the original set. To cope with this, we can fill
    # the missing by replacing them
    # with the latest available price for each instrument.
    close = close.fillna(method='ffill')
    return close

In [146]:
def get_rolling_window(window,stock):
    # Calculate the window days moving averages of the closing prices
    short_rolling = stock.rolling(window=window).mean()
    return short_rolling

In [147]:
def get_rolling_plot(rolling,stock,window,ticker,panel_data,start,end):
    # Plot everything by leveraging the very powerful
    # matplotlib package
    fig, ax = plt.subplots(2,1,figsize=(13,8))
    #Calculate full sample mean
    full_sample_mean = stock.mean()
    
    ax[0].plot(stock.index, stock, label=ticker)
    ax[0].plot(rolling.index, rolling, label=str(window)+' days rolling')
    ax[0].axhline(full_sample_mean,linestyle='--',color='red',label='Full Sample Mean')
    
    ax[0].set_xlabel('Date')
    ax[0].set_ylabel('Adjusted closing price ($)')
    ax[0].legend()
    
    ax[1].plot(panel_data.loc[start:end,].index, panel_data.loc[start:end],label='Volume Traded')
    
    ax[1].set_ylabel('Volume Traded')
    ax[1].xaxis.set_major_formatter(my_year_month_fmt);

In [148]:
def get_ema_short(close,span):
    # Using Pandas to calculate a 20-days span EMA. 
    #adjust=False specifies that we are interested in the recursive calculation mode.
    ema_short = close.ewm(span=span, adjust=False).mean()
    return ema_short

In [149]:
def ema_plot(close,start,end,ticker,ema_short,span,panel_data):
    # Taking the difference between the prices and the EMA timeseries
    trading_positions_raw = close - ema_short
    trading_positions_raw.tail()

    # Taking the sign of the difference to determine whether the price or the EMA is greater and then multiplying by 1/3
    trading_positions = trading_positions_raw.apply(np.sign) * 1/2
    trading_positions.tail()

    # Lagging our trading signals by one day.
    trading_positions_final = trading_positions.shift(1)
    
    fig, (ax,ax2) = plt.subplots(2,1,figsize=(13,8))

    ax.plot(close.loc[start:end,].index, close.loc[start:end], label=ticker)
    ax.plot(ema_short.loc[start:end,].index, ema_short.loc[start:end], label ='Span '+str(span)+'-days EMA')
    ax.legend(loc='best')
    ax.set_ylabel('Price in $')
    ax.xaxis.set_major_formatter(my_year_month_fmt)
    
    ax2.plot(trading_positions_final.loc[start:end,].index, trading_positions_final.loc[start:end],label='Trading position')
    
    ax2.set_ylabel('Trading position-'+str(span))
    ax2.xaxis.set_major_formatter(my_year_month_fmt)

In [150]:
def cycle_indicator_plot(close,panel_data,start,end,ticker):
    sine, leadsine = talib.HT_SINE(panel_data["Adj Close"])
    integer = talib.HT_TRENDMODE(panel_data["Adj Close"])
    
    fig, (ax_stocktrend,ax_trendmode,ax_ht_sine) = plt.subplots(3,1,figsize = (13,10))
    
    ax_stocktrend.plot(close.loc[start:end,].index, close.loc[start:end], label=ticker)
    ax_stocktrend.legend(loc="best")
    ax_stocktrend.set_ylabel('Price in $')
    
    ax_trendmode.plot(integer,color = "#63abdb")
    ax_trendmode.set_ylabel("Hilbert Transform - Trend vs Cycle Mode")
    
    ax_ht_sine.plot(panel_data.index,sine, color = "#63abdb",label="Sine")
    ax_ht_sine.plot(panel_data.index, leadsine, color="#63abdb", dashes=[2, 2, 2, 2],label="Leadsine")
    ax_ht_sine.legend(loc="best")
    ax_ht_sine.set_ylabel('Hilbert Transform - SineWave')
    

In [151]:
def momentum_plot(panel_data):
    rsi = talib.RSI(panel_data["Adj Close"])
    wr = talib.WILLR(panel_data["High"],panel_data["Low"],panel_data["Adj Close"], timeperiod=14)
    
    fig, (ax_rsi,ax_wr) = plt.subplots(2,1,figsize=(13,8))
    
    ax_rsi.plot(panel_data.index, [70] * len(panel_data.index), label="overbought")
    ax_rsi.plot(panel_data.index, [30] * len(panel_data.index), label="oversold")
    ax_rsi.plot(panel_data.index, rsi, label="Relative Strength Indicator (RSI)")
    ax_rsi.legend(loc='best')
    ax_rsi.plot(panel_data["Adj Close"])
    
    ax_wr.plot(panel_data.index, [-20] * len(panel_data.index))
    ax_wr.plot(panel_data.index, [-80] * len(panel_data.index))
    ax_wr.plot(panel_data.index, wr,color="#2413bd",label="Williams %R (WILLR)")
    ax_wr.legend(loc='best')
    ax_wr.plot(panel_data["Adj Close"])
    

In [152]:
def bollinger_bands(panel_data):
    upperband, middleband, lowerband = talib.BBANDS(panel_data["Adj Close"], timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    
    fig,ax = plt.subplots(figsize=(13,4))
    
    ax.plot(panel_data.index,upperband,label="Upperband",linewidth=1)
    ax.plot(panel_data.index,middleband,label="Middleband",linewidth=1)
    ax.plot(panel_data.index,lowerband,label="Lowerband",linewidth=1)
    ax.legend(loc="best")
    ax.xaxis.set_major_formatter(my_year_month_fmt)

In [238]:
def trend_strategy(panel_data,threshold):
    """
    This trend strategy is based on both a two-month (i.e., 42 tradingdays) and a one-year 
    (i.e., 252 trading days) trend (i.e., the moving average of the index level for the respective period
    
    The rule to generate trading signals is the following:
        Buy signal (go long):
            the 42d trend is for the first time SD points above the 252d trend.
        Wait (park in cash)
            the 42d trend is within a range of +/– SD points around the 252d trend.
        Sell signal (go short)
            the 42d trend is for the first time SD points below the 252d trend
    """
    
    stock_sd= pd.DataFrame()
    stock_sd['Close'] = panel_data["Adj Close"]
    stock_sd['42d'] = np.round(stock_sd['Close'].rolling(42).mean(), 2)
    stock_sd['252d'] = np.round(stock_sd["Close"].rolling(252).mean(), 2)
    
    stock_sd["42-252"] = stock_sd["42d"]-stock_sd["252d"]
    
    SD = threshold
    stock_sd['Regime'] = np.where(stock_sd['42-252'] > SD, 1, 0)
    stock_sd['Regime'] = np.where(stock_sd['42-252'] < -SD, -1, stock_sd['Regime'])
    #print(stock_sd['Regime'].value_counts())    
    
    #performance of regime strategy
    stock_sd['Market'] = np.log(stock_sd['Close'] / stock_sd['Close'].shift(1))
    stock_sd['Strategy'] = stock_sd['Regime'].shift(1) * stock_sd['Market']
    #.plot(grid=True,
# figsize=(8, 5))
    
    fig, ax = plt.subplots(3,1,figsize=(13,8))
    ax[0].plot(stock_sd["Close"],label="Close")
    ax[0].plot(stock_sd["42d"],label="42d")
    ax[0].plot(stock_sd["252d"],label="252d")
    ax[0].xaxis.set_major_formatter(my_year_month_fmt)
    ax[0].legend(loc="best")
    
    ax[1].plot(stock_sd["Regime"],linewidth=1.5)
    ax[1].axis(ymin=-1.1,ymax=1.1)
    ax[1].xaxis.set_major_formatter(my_year_month_fmt)
    
    ax[2].plot(stock_sd['Market'].cumsum().apply(np.exp),label="Market",color="blue")
    ax[2].plot(stock_sd['Strategy'].cumsum().apply(np.exp),label = "Strategy",color="green")
    ax[2].legend(loc="best")
    ax[2].xaxis.set_major_formatter(my_year_month_fmt)

In [239]:
type_of_plots =["Exponential Moving Average (EMA)","Simple Moving Average",\
                "Momentum", "Cycle Indicators","Bollinger Bands","Trend Strategy"]

In [240]:
def typeOfPlot(close,start,end,ticker,span,plot,window,stock,threshold,panel_data):
    if plot == "Exponential Moving Average (EMA)":
        ema_short = get_ema_short(close,span)
        ema_plot(close,start,end,ticker,ema_short,span,panel_data)
    elif plot == "Simple Moving Average":
        rolling = get_rolling_window(window,stock)
        get_rolling_plot(rolling,stock,window,ticker,panel_data,start,end)
    elif plot=="Momentum":
        momentum_plot(panel_data)
    elif plot == "Cycle Indicators":
        cycle_indicator_plot(close,panel_data,start,end,ticker)
    elif plot == "Bollinger Bands":
        bollinger_bands(panel_data)
    elif plot =="Trend Strategy":
        trend_strategy(panel_data,threshold)
    else:
        plt.gca()

In [241]:
sector_widget = widgets.Dropdown(options = sorted(list(stock_df.columns)),\
                description='Sector:',disabled=False)
ticker_widget = widgets.Dropdown(options=\
                sorted(list(stock_df['Aggressive Small Cap Stocks'].dropna())),\
                description='Ticker:',disabled=False)
plot_widget = widgets.Dropdown(options = sorted(type_of_plots),description='Plot:',\
              disabled=False)
start_date = widgets.DatePicker(description='Start Date',disabled=False,\
                                value=datetime.date(2018,1,1))
end_date = widgets.DatePicker(description='End Date',disabled=False,\
                              value=datetime.date.today())
window_widget = widgets.IntText(value=10,min=5,max=500,step=1,description='Window:',\
                                disabled=False)
threshold = widgets.IntText(value=20,min=5,max=500,step=1,description='Threshold:',\
                                disabled=False)
span_widget = widgets.IntText(value=10,min=5,max=500,step=1,description='Ema Span:',\
                              disabled=False)
                                 
#Define a function that updates the content of ticker based on what we select for
#sector

def update(*args):
    ticker_widget.options = sorted(list(stock_df[sector_widget.value].dropna()))
sector_widget.observe(update,'value')


def plots(sector,ticker,start,end,window,span,threshold,plot):
    global panel_data
    global stock
    
    try:
        # User pandas_reader.data.DataReader to load the desired data
        panel_data = data.DataReader(ticker, 'yahoo', start,end)

    
        close = get_data(panel_data,start,end)
        stock = close.loc[:]
        
        typeOfPlot(close,start,end,ticker,span,plot,window,stock,threshold,panel_data)
    
    except:
        plt.gca();
    
interact(plots,sector=sector_widget,ticker=ticker_widget,start = start_date,end=end_date,window=window_widget,\
        span=span_widget,threshold=threshold,plot=plot_widget)

interactive(children=(Dropdown(description='Sector:', options=('Aggressive Small Cap Stocks', 'Basic Materials…

<function __main__.plots(sector, ticker, start, end, window, span, threshold, plot)>

In [212]:
# from selenium import webdriver
# from bs4 import BeautifulSoup as bs

# DRIVER_PATH = 'C:\Windows\chromedriver.exe'
# driver = webdriver.Chrome(executable_path=DRIVER_PATH)
# link="https://finance.yahoo.com/screener/predefined/ms_Industrials?count=100&offset={}"
# #get html
# i = 0
# soup = []
# while i < 500:
#     driver.get(link.format(i))
#     soup.append(bs(driver.page_source,'html.parser'))
#     i += 100
# len(soup)

In [164]:
# industrials_stocks = []
# for w in soup:
#     a = w.find_all('a',class_='Fw(600) C($linkColor)')
#     for b in a:
#         industrials_stocks.append(b.text)
# print(industrials_stocks)